# Honors Thesis
## An Exploratory Study of Biometric Gait Recognition
## Compiled by David Kartchner
## November 30, 2016

In [ ]:
import numpy as np
import pandas as pd
from __future__ import division
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
import matplotlib
import pywt
from scipy.fftpack import fft, ifft
from scipy.interpolate import interp1d
import seaborn as sns
from scipy import stats
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (14.0, 7.0)
pd.set_option('expand_frame_repr', True)
pd.set_option('max_rows',200)

## Define some useful helper functions

In [ ]:
def get_rid_of_useless_data(data, col="A_tot", trim_ends=False):
    data = data.copy().reset_index()
    rate = int(data.Samplerate[0])
    
    # Smooth data by taking wavelet transform
    data["smoothed"] = data["A_tot"]#wavelet_smooth(data, [col])
    
    #Find standard deviation for 1 second rolling windows of smoothed data
    new_col = col+"_rolling_std"
    data[new_col] = data['smoothed'].rolling(center=True, 
                                             window=int(rate), 
                                             min_periods=rate//2).std()
    
    #Drop segments of data where change in acceleration is too small
    std_dev = data[new_col].mean()
    data.loc[data[new_col]<std_dev/20,"smoothed"] = -1
    
    #When acceleration is too low, drop everything in the window where it was too small.
    data['smoothed'] = data['smoothed'].rolling(center=True, 
                                                window=int(rate), 
                                                min_periods=rate//2).min()
    
    # Drop first 5 seconds and last 5 seconds for good measure
    data = data.loc[data['smoothed']>0,:]
    if trim_ends:
        return data.loc[rate*5:(data.shape[0]-rate*5),["A_tot","Samplerate", "JointID"]].reset_index()
    else:
        return data.loc[:,["A_tot","Samplerate", "JointID"]].reset_index()

In [ ]:
def clean_data(data, downsample=True):
    #Generate a column that corresponds to the person and the activity
    #Note that JointID//47 gives the activity and JointID%47 gives the person
    data["JointID"] = 47*data["ActivityID"] + data["PersonID"]
    
    #Drop "microwalk" activity values since they don't matter for our classification tests.
    data = data[data.ActivityID<=7]
    
    #All columns have same type of activity, so we drop ActivityID
    data.drop(["SessionID", "TimeDelta", "DeviceID", "ActivityID","PersonID"], 1, inplace=True)
    
    if downsample:
        #Downsample the data so that the sample rate is always 50
        #First, separate data in to rate groups
        rate_groups = data.groupby("Samplerate")
        rate_50 = rate_groups.get_group(50)
        rate_400 = rate_groups.get_group(400)

        #Now parse the data by individual and average each 8 consecutive observations
        downsampled_groups = []
        person_groups = rate_400.groupby("JointID")
        for person in person_groups.groups:
            group = person_groups.get_group(person)
            group["Samplerate"] = 50
            downsampled_groups.append(group.groupby(lambda x: x//8).mean())

        #Now that each individual has been downsampled, join everything back together.
        downsampled_400 = pd.concat(downsampled_groups)
        data=pd.concat([downsampled_400, rate_50])
        
    
    #We add a column for the total acceleration at each point
    data['A_tot'] = np.sqrt(data["Ax"]**2 + data["Ay"]**2 + data["Az"]**2)
    
#     grouped_data = data.groupby("JointID")
#     chunks = []
#     for group in grouped_data.groups:
#         temp = grouped_data.get_group(group)
#         if temp.shape[0]>5000:
#             chunk = get_rid_of_useless_data(temp, trim_ends=True)
#         else:
#             chunk = get_rid_of_useless_data(temp, trim_ends=False)
#         chunks.append(chunk)
        
#     data = pd.concat(chunks)
    
    #Classify slow walk and fast walk as simply "walking"
    #data.replace(to_replace={'ActivityID':{2:1, 3:1}},inplace=True)
    
    
    return data.loc[:,["A_tot","Samplerate", "JointID"]]

In [ ]:
def fourier_smooth(data, cols):
    # Select the 10% of coefficients corresponding to the lowest frequencies
    # (The above strategy works well for data sampled 400 times/second, not sure about 50 times/second)
    num_coefs = data.shape[0]//20
    for col in cols:
        # Take DFT
        dft = fft(data[col])
        
        # Select low frequcney coeffs from start and end of DFT
        start = dft[:num_coefs]
        end = dft[-num_coefs:]
        
        # Set all other coefs to 0
        dft = np.zeros_like(dft)
        dft[:num_coefs] = start
        dft[-num_coefs:] = end
        
        # Take inverse DFT to get smoothed signal
        smoothed_signal = ifft(dft)
        data["FourierSmooth_"+col] = smoothed_signal
        
    return data

def rolling_smooth(data, cols,  min_samples=4):
    stdev = data.Samplerate[0]/10.
    for col in cols:
        data["RollSmooth_"+col] = data[col].rolling(center=True, 
                                                    window=int(.4*data.loc[0,'Samplerate']), 
                                                    min_periods=min_samples, win_type='gaussian').mean(std=stdev)
    return data



def wavelet_smooth(data, cols, num_discard=4, wave_type='db4'):
    for col in cols:
        #Take wavelet transform and throw away high frequency coefficients
        wavelets = pywt.wavedec(data[col], wave_type)
        for i in xrange(-num_discard,0):
            wavelets[i] = np.zeros_like(wavelets[i])
        denoised_signal = pywt.waverec(wavelets,wave_type)
        
        try:
            data["WavSmooth_"+col] = denoised_signal
        except:
            data["WavSmooth_"+col] = denoised_signal[:-1]
    return data["WavSmooth_"+col]

In [ ]:
def extract_gait_cycles(data, normalize=False, raw=False):
    # Smooth the total acceleration to find cycle starting and ending points
    data = data.copy().reset_index()
    data = rolling_smooth(data,["A_tot"])
    rate = data.Samplerate[0]
    
    # Find "salient" minima and label them
    data["Min_Salience"] = data.loc[:,"RollSmooth_A_tot"].rolling(center=True, 
                                                                  window=int(1.2*rate), 
                                                                  min_periods=10).min()
    data["salience_points"] = (data.loc[:,"Min_Salience"] == data.loc[:,"RollSmooth_A_tot"]).astype(int)
    
    # Enumerate different cycles and extract them into a list
    data["cycle_label"] = data.salience_points.cumsum()
    
    if raw:
        return data
    
    gait_cycles = data.groupby("cycle_label")
    cycles = [gait_cycles.get_group(grp)["A_tot"].as_matrix().flatten() for grp in gait_cycles.groups][1:-1]
    
    # Get labels for our data
    labels = np.array([data.loc[0,"JointID"]]*len(cycles))
    
    # Calculate potentially relevant features
    mins = np.array([cycle.min() for cycle in cycles])
    maxs = np.array([cycle.max() for cycle in cycles])
    stds = np.array([np.std(cycle) for cycle in cycles])
    means = np.array([cycle.mean() for cycle in cycles])
    cycle_lengths = np.array([len(cycle) for cycle in cycles])
                                                                                                                  
                                                                                                                  
    # Normalize features if desired
    if normalize:
        cycles = [(cycle-mini)/(maxi-mini)for cycle, mini, maxi in zip(cycles, mins, maxs)]
    
    # Get interpolating polynomials and get standard set of points for data
    polynomials = [interp1d(np.linspace(0,1,len(cycle)),cycle) for cycle in cycles]
    interp_points = np.array([poly(np.linspace(0,1,51)) for poly in polynomials])
    
    interp_df = pd.DataFrame(interp_points) 
    labels_df = pd.DataFrame(labels, columns=["Label"])
    feats_df = pd.DataFrame(np.array([mins, maxs, stds, means, cycle_lengths]).transpose(),
                            columns=["Min","Max","Std","Mean","Cycle_Length"])

    mask = (feats_df["Cycle_Length"]>rate*.8)&(feats_df["Cycle_Length"]<rate*1.5)
    interp_df = interp_df.loc[mask].reset_index(drop=True)
    feats_df = feats_df[mask].reset_index(drop=True)
    labels_df = labels_df[mask].reset_index(drop=True)
    
#     print labels_df.head(), feats_df.head(), interp_df.head()
    # Trim down data frames with too many rows for balanced training set
    if labels_df.shape[0]>150:
        
        rows = np.random.choice(labels_df.shape[0]-1, size=150, replace=False)
        print rows.min()
        print rows.max()
        print interp_df.shape
        
        interp_df = interp_df.loc[rows]
        feats_df = feats_df.loc[rows]
        labels_df = labels_df.loc[rows]
    
    
    return interp_df, labels_df, feats_df

In [ ]:
accel_data = clean_data(pd.read_csv('ProjectData.csv'))
group1 = accel_data.groupby("JointID").get_group(53)
group2 = accel_data.groupby("JointID").get_group(55)

In [ ]:
data = extract_gait_cycles(group2, raw=True)
new_dat = data.loc[(data['cycle_label']>200)&(data['cycle_label']<204),["A_tot","cycle_label","salience_points","RollSmooth_A_tot"]]
start_indices = new_dat[new_dat['salience_points']==1].index.tolist()
ax1 = new_dat[["A_tot"]].plot(x=np.linspace(new_dat.index.min()/50, new_dat.index.max()/50,new_dat.shape[0]))
plt.title("Gait Cycle Segmentation", fontsize=24)
plt.ylabel("Acceleration", fontsize=18)
plt.xlabel("Time (seconds)", fontsize=18)
iters=0
for i in start_indices:
    if iters==0:
        ax1.axvline(x=i/50, color='r', linestyle='--', label="Cycle Starts")
    else:
        ax1.axvline(x=i/50, color='r', linestyle='--')
    iters += 1
plt.legend()
plt.show()

In [ ]:
print data.columns
data = extract_gait_cycles(group1, raw=True)
new_dat = data.loc[(data['cycle_label']>200)&(data['cycle_label']<225),["A_tot","cycle_label","salience_points","RollSmooth_A_tot"]]
start_indices = new_dat[new_dat['salience_points']==1].index.tolist()
ax1 = new_dat[["A_tot", "RollSmooth_A_tot"]].plot()
for i in start_indices:
    ax1.axvline(x=i, color='r', linestyle='--')

In [ ]:
print data.columns
data = extract_gait_cycles(group1, raw=True)
new_dat = data.loc[(data['cycle_label']>70)&(data['cycle_label']<76),["A_tot","cycle_label","salience_points","RollSmooth_A_tot"]]
start_indices = new_dat[new_dat['salience_points']==1].index.tolist()
ax1 = new_dat[["A_tot"]].plot(x=np.linspace(new_dat.index.min()/50, new_dat.index.max()/50,new_dat.shape[0]))
plt.title("Gait Cycle Segmentation", fontsize=24)
plt.ylabel("Acceleration", fontsize=18)
plt.xlabel("Time (seconds)", fontsize=18)
iters=0
for i in start_indices:
    if iters==0:
        ax1.axvline(x=i/50, color='r', linestyle='--', label="Cycle Starts")
    else:
        ax1.axvline(x=i/50, color='r', linestyle='--')
    iters += 1
plt.legend()
plt.show()

In [ ]:
def template_split(data, label_col):
    #Randomly assigns rows to training and testing data
    cols = data.columns
    grouped_data = data.groupby(label_col)
    train, test = [], []
    for group in grouped_data.groups:
        data = grouped_data.get_group(group).reset_index()
        
        num_rows = data.shape[0]

        test_rows = np.random.choice(num_rows, size=np.floor(.3*num_rows), replace=False)
        test_data = data.loc[test_rows]
        train_mask = np.ones(num_rows, dtype=bool)
        train_mask[test_rows] = 0
        train_data = data.loc[train_mask]
        test.append(test_data.loc[:,cols])
        train.append(train_data.loc[:,cols])
    test = pd.concat(test)
    train = pd.concat(train)
#     print train.columns
    return train, test

In [ ]:
groups = accel_data.groupby(by="JointID", as_index=False)
data_groups = [groups.get_group(i) for i in groups.groups]

# Extract gait cycles from data
interp_pts = []
labels = []
other_feats = []
for group in data_groups:
    pts, labs, feats = extract_gait_cycles(group, normalize=False)
    interp_pts.append(pts)
    labels.append(labs)
    other_feats.append(feats)

In [ ]:
# Merge interpolated points together into single data frame
interp_points = pd.concat(interp_pts).reset_index().loc[:,[i for i in range(51)]]

new_labels = pd.concat(labels).reset_index().loc[:,["Label"]]

new_other_feats = pd.concat(other_feats).reset_index().loc[:,["Min","Max","Std","Mean","Cycle_Length"]]
interp_points = pd.concat([interp_points, new_labels],axis=1)
full_feats = pd.concat([interp_points, new_other_feats], axis=1)

In [ ]:
# Split data by train and test
interp_train, interp_test = template_split(interp_points, label_col="Label")
full_train, full_test = template_split(full_feats, label_col="Label")

In [ ]:
# Split data by train and test
interp_train, interp_test = template_split(interp_points, label_col="Label")
full_train, full_test = template_split(full_feats, label_col="Label")
one_norm_accuracies = []
two_norm_accuracies = []
for i in xrange(10):
    interp_train, interp_test = template_split(interp_points, label_col="Label")
    uniform_KNN = KNeighborsClassifier(n_neighbors=1, n_jobs=-1, p=1).fit(interp_train.drop("Label",1),
                                                             interp_train["Label"])
    uniform_accuracy = accuracy_score(uniform_KNN.predict(interp_test.drop("Label",1)),
                                                             interp_test["Label"])
    
    weighted_KNN = KNeighborsClassifier(n_neighbors=1, n_jobs=-1,
                                         p=2).fit(interp_train.drop("Label",1),
                                                             interp_train.Label)
    weighted_accuracy = accuracy_score(weighted_KNN.predict(interp_test.drop("Label",1)),
                                                             interp_test["Label"])
#     print "{0} Neighbors, Uniform: {1}".format(i, uniform_accuracy)
#     print "{0} Neighbors, Weighted: {1}\n".format(i, weighted_accuracy)
    one_norm_accuracies.append(uniform_accuracy)
    two_norm_accuracies.append(weighted_accuracy)
print "Mean KNN 1-norm Accuracy:", np.mean(one_norm_accuracies)
print "Mean KNN 2-norm Accuracy:", np.mean(two_norm_accuracies)

In [ ]:
def agg_accuracy(classifier=KNeighborsClassifier(n_neighbors=1, n_jobs=-1, p=1)):
    total_acc = []
    for i in xrange(50):
        interp_train, interp_test = template_split(interp_points, label_col="Label")
        model = classifier.fit(interp_train.drop("Label",1),interp_train["Label"])


        interp_test["one_pred"] = model.predict(interp_test.drop("Label",1))
    #     interp_test["two_pred"] = weighted_KNN.predict(interp_test.drop("Label",1))

        accuracies = []
        grouped = interp_test.groupby("Label")
        for i in grouped.groups:
            group = grouped.get_group(i)
            n = min([group.shape[0], 11])
            rows = np.random.choice(group.index, size=n, replace=False)
            mode, count = stats.mode(group.loc[rows,"one_pred"])
            if mode==group.Label.as_matrix()[0] and count/n>.5:
                accuracies.append(1)
            else:
                accuracies.append(0)
        total_acc.append(np.mean(accuracies))

    print np.mean(total_acc)

    #     print "{0} Neighbors, Uniform: {1}".format(i, uniform_accuracy)
    #     print "{0} Neighbors, Weighted: {1}\n".format(i, weighted_accuracy)

print "KNN Aggregated Accuracy"
agg_accuracy()

print "Random Forest Aggregated Accuracy"
agg_accuracy(RandomForestClassifier(n_estimators=100, n_jobs=-1))

In [ ]:
rf_accuracies = []
for i in xrange(5):
    rf_acc = random_forest_model(interp_train["Label"],interp_train.drop("Label",1),
                    interp_test["Label"], interp_test.drop("Label",1),
                    n_runs=5, plot=False, model_type="Person")[0]
    rf_accuracies.append(rf_acc)
print np.mean(rf_accuracies)

In [ ]:
XGB_model(interp_train["Label"],interp_train.drop("Label",1),
                    interp_test["Label"], interp_test.drop("Label",1),
                    n_runs=10, plot=False, model_type="Person")

In [ ]:
print interp_train.shape
print interp_test.shape

In [ ]:
accel_data = clean_data(pd.read_csv('ProjectData.csv'))
labels = accel_data.JointID.unique()
labels=labels[labels>=60]
labels
grouped = accel_data.groupby("JointID")
for i in labels:
    data = grouped.get_group(i)
    data.A_tot.plot()
    plt.show()

In [ ]:
accel_data = clean_data(pd.read_csv('ProjectData.csv'))
grouped = accel_data.groupby("JointID")
group1=grouped.get_group(53)


In [ ]:
group1.plot(x=np.linspace(group1.index.min()/50, group1.index.max()/50,group1.shape[0]), y="A_tot", legend=False)
plt.xlabel("Time (Seconds)", fontsize=18)
plt.ylabel("Acceleration", fontsize=18)
plt.title("Acceleration Readings over Time", fontsize=24)

In [ ]:
def feature_extraction(data, split_var="JointID"):
    #When performing activity classification, we still split our data based on person and activity, 
    # but then label based on activity
    if split_var=="ActivityID":
        grouped = data.groupby(["JointID"])
    else:
        grouped = data.groupby([split_var])
    x_s = []
    labels = []
    
    #Calculate histogram bins to use later for features
    histogram_bins = np.linspace(0,1,11)
       
    for user, features in grouped:
        print user
        #Get feature attributes on 3 second samples from each individual
        #We assume no overlap between samples
        sample_rate = features["Samplerate"].as_matrix()[0]
        num_samples = features.count().as_matrix()[0]
        N_3_sec = int(sample_rate*3)
        
        #Normalize data by centering each individual at 0. 
        #features["A_tot"] -= features["A_tot"].mean()
        
        #Extract as many full samples as we can
        n_extracts = int(num_samples//N_3_sec)
        for i in xrange(n_extracts):
            attributes = []
            
            
            labels.append(features[split_var].as_matrix()[0])

            subset = features[N_3_sec*i:N_3_sec*(i+1)]

            
#             attributes.append(subset.JointID.as_matrix()[0])
            attributes.append(subset["A_tot"].mean())
            attributes.append(subset["A_tot"].std())
            attributes.append(subset["A_tot"].max())
            attributes.append(subset["A_tot"].min())
            attributes.append(subset["A_tot"].max() - subset["A_tot"].min())
            
            #Calculate RMS of total acceleration of each sample
            attributes.append(np.sqrt(np.sum(np.square(subset["A_tot"])/N_3_sec)))
            
            #Calculate signal entropy
            dft = fft(subset.A_tot)
            power = np.abs(dft)**2/len(dft)
            p_i = power/np.sum(power)
            pse = -(np.sum(p_i*np.log(p_i)))
            attributes.append(pse)
            
            #Calculate histogram distribution of of samples in dataset
            normed = (subset["A_tot"]-subset.A_tot.min())/(subset.A_tot.max()-subset.A_tot.min())
            for k in np.histogram(normed, bins=histogram_bins)[0]:
                attributes.append(k)
                
            #Calculate total number of sign changes in the subset
            subset.A_tot = subset.A_tot-subset.A_tot.mean()
            attributes.append(np.sum(np.abs((np.sign(subset["A_tot"][:-1]).as_matrix() - np.sign(subset["A_tot"][1:]).as_matrix())/2)))
            
            
            x_s.append(attributes)

    dat = pd.DataFrame(np.hstack((np.array(labels).reshape((len(labels),1)),np.array(x_s))),
                       columns=["Label", "Mean","Std","Max","Min","Range","RMS","FDE"]+[str(i) for i in range(10)]+["Sign"])
    return dat
    
        

In [ ]:
def split_train_test_data(labels, features):
    #Randomly assigns rows to training and testing data
    
    num_rows = features.shape[0]

    test_rows = np.random.choice(num_rows, size=np.floor(.3*num_rows), replace=False)
    test_features = features[test_rows]
    test_labels = labels[test_rows]
    train_mask = np.ones(num_rows, dtype=bool)
    train_mask[test_rows] = 0
    train_features = features[train_mask]
    train_labels = labels[train_mask]
    return train_labels, train_features, test_labels, test_features

In [ ]:
def random_forest_model(tr_labels, tr_features, te_labels, te_features, n_runs=10, plot=True, model_type="Person"):
    correctly_classified = np.zeros(n_runs)
    run_importances = np.zeros((n_runs,10))
    activity_classification = np.zeros(7)
    
    for i in xrange(n_runs):
        print i+1
        #Train a random forest and predict the number correct
        rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
        rf.fit(tr_features, tr_labels)
        predicted_labels = rf.predict(te_features)
        correct_proportion = 1.-len(np.where(predicted_labels != te_labels)[0])/te_labels.shape[0]
        print "Correct Proportion:", correct_proportion
        print te_labels
        print accuracy_score(predicted_labels, te_labels)
        #print "Proportion correctly classified: ", correct_proportion
        correctly_classified[i] = correct_proportion

        #Calculate feature importances
        feat_importances = rf.feature_importances_
        hist_importances = np.sum(feat_importances[8:-1])
        importances = np.empty(10)
        importances[:8] = feat_importances[:8]
        importances[9] = hist_importances
        importances[8] = feat_importances[9]
        run_importances[i,:] = importances
    
    activity_classification /= n_runs
    correct = np.mean(correctly_classified)
    final_importances = np.mean(run_importances, axis=0)
    print "Proportion correctly classified: ", correct
    
    if plot==True:
        pos = np.arange(len(final_importances))+.5
        #plt.barh(pos, importances, color='c')
        plt.xlabel("Feature Importance", fontsize=18)
        plt.ylabel("Feature", fontsize=18)
        plt.title("Random Forest Feature Importances", fontsize=24)
        labels = ["Mean", "Standard Deviation",'Min', 'Max',
                  "Value Range","Root Mean Square","Sign Changes",
                  "Frequency Domain Entropy",'Sign Changes',"Histogram Distribution"]
        
        s = pd.Series(final_importances, index=labels)
        mycolors = ['b', 'g', 'r', '#FFA500','#800080','c', '#00FF7F', "m", '#FF7F50','y']
        s.plot(kind='barh', color=mycolors)
        plt.show()
        
        
    return correct, final_importances

In [ ]:
def XGB_model(tr_labels, tr_features, te_labels, te_features, n_runs=10, plot=True, model_type="Person"):
    correctly_classified = np.zeros(n_runs)
    run_importances = np.zeros((n_runs,9))
    activity_classification = np.zeros(7)
    
    for i in xrange(n_runs):
        print i+1
        #Train a random forest and predict the number correct
        rf = XGBClassifier(n_estimators=100)
        rf.fit(tr_features, tr_labels)
        predicted_labels = rf.predict(te_features)
        correct_proportion = 1.-len(np.where(predicted_labels != te_labels)[0])/te_labels.shape[0]
        #print "Proportion correctly classified: ", correct_proportion
        correctly_classified[i] = correct_proportion

#         #Calculate feature importances
#         feat_importances = rf.feature_importances_
#         hist_importances = np.sum(feat_importances[8:])
#         importances = np.empty(9)
#         importances[:8] = feat_importances[:8]
#         importances[8] = hist_importances
#         run_importances[i,:] = importances
    
    activity_classification /= n_runs
    correct = np.mean(correctly_classified)
    final_importances = np.mean(run_importances, axis=0)
    print "Proportion correctly classified: ", correct
    
    if plot==True:
        pos = np.arange(len(final_importances))+.5
        #plt.barh(pos, importances, color='c')
        plt.xlabel("Feature Importance")
        labels = ["Mean", "Standard Deviation",'Min', 'Max',
                  "Value Range","Root Mean Square","Sign Changes",
                  "Frequency Domain Entropy","Histogram Distribution"]
        
        s = pd.Series(final_importances, index=labels)
        mycolors = ['b', 'g', 'r', '#FFA500','#800080','c', '#00FF7F', "m", '#FF7F50']
        
        s.plot(kind='barh', color=mycolors)
        plt.show()
        
        
    return correct, final_importances

In [ ]:
def template_split(data, label_col="JointID"):
    #Randomly assigns rows to training and testing data
    cols = data.columns
    grouped_data = data.groupby(label_col)
    train, test = [], []
    for group in grouped_data.groups:
        data = grouped_data.get_group(group).reset_index()
        
        num_rows = data.shape[0]

        test_rows = np.random.choice(num_rows, size=np.floor(.3*num_rows), replace=False)
        test_data = data.loc[test_rows]
        train_mask = np.ones(num_rows, dtype=bool)
        train_mask[test_rows] = 0
        train_data = data.loc[train_mask]
        test.append(test_data.loc[:,cols])
        train.append(train_data.loc[:,cols])
    test = pd.concat(test)
    train = pd.concat(train)
#     print train.columns
    return train, test

In [ ]:
data = feature_extraction(accel_data, split_var="JointID")
print "Features Created"
train, test = template_split(data, label_col="Label")
rain_l = train.Label
train_f = train.drop("Label",1)
test_l = train.Label
test_f = train.drop("Label",1)

In [ ]:
train.describe()

In [ ]:
test.describe()
np.unique(test.Label, return_counts=True)

In [ ]:
def person_model(train, test, find_tree_nums=False):
    #Train a model that recognized people from only walking data
    


    train_l = train.Label
    train_f = train.drop("Label",1)
    print train_f.head()
    test_l = train.Label
    test_f = train.drop("Label",1)
    print test_f.head()
    print "Data Split"
    if find_tree_nums:
        print "Running Optimal Tree Model"
        find_optimal_tree_num(train_l, train_f, test_l, test_f)
    print "Running Random Forest Trials"
    correct, feature_importances = random_forest_model(train_l, train_f, test_l, test_f, n_runs=10, plot=True)
    
person_model(train, test, find_tree_nums=False)

In [ ]:
person_model(find_tree_nums=False)

In [ ]:
# full_data = clean_data(pd.read_csv('ProjectData.csv'), downsample=False)
accel_data = clean_data(pd.read_csv('ProjectData.csv'))

In [ ]:
accel_data.shape

In [ ]:
full_group1 = full_data.groupby(by="JointID", as_index=False).get_group(53)
group1 = accel_data.groupby(by="JointID", as_index=False).get_group(53)
group1.shape

In [ ]:
start_plot = 201000
diff = 1200
plt.title("Comparison of Regular and Downsampled Acceleration")
plt.ylabel("Acceleration")
plt.xlabel("Time (Seconds)")
plt.plot(np.linspace(0, diff/400, diff+1),full_group1.loc[start_plot:start_plot+diff, "A_tot"], label="Original Signal")
plt.plot(np.linspace(0, diff/400, diff//8+1),group1.loc[start_plot//8:(start_plot+diff)//8, "A_tot"], label="Downsampled Signal")
plt.legend(loc="best")
plt.show()

## Extract gait cycles from each group in preparation for template based learning approach

In [ ]:

labels.describe()

In [ ]:
group1.A_tot.plot()

In [ ]:
print len(groups.groups)

In [ ]:
other_feats[0].describe()

In [ ]:
group1 = get_rid_of_useless_data(accel_data.groupby(by="JointID", as_index=False).get_group(53))
group1.shape

In [ ]:
print data.columns
data = extract_gait_cycles(group1, raw=True)
new_dat = data.loc[(data['cycle_label']>200)&(data['cycle_label']<205),["A_tot","cycle_label","salience_points","RollSmooth_A_tot"]]
start_indices = new_dat[new_dat['salience_points']==1].index.tolist()
ax1 = new_dat[["A_tot", "RollSmooth_A_tot"]].plot()
for i in start_indices:
    ax1.axvline(x=i, color='r', linestyle='--')

In [ ]:
group1 = accel_data.groupby(by="JointID", as_index=False).get_group(53)
data = extract_gait_cycles(group1, raw=True)
new_dat = data.loc[(data['cycle_label']>212)&(data['cycle_label']<217),["A_tot","cycle_label","salience_points","RollSmooth_A_tot"]]
start_indices = new_dat[new_dat['salience_points']==1].index.tolist()
ax1 = new_dat[["A_tot", "RollSmooth_A_tot"]].plot()
for i in start_indices:
    ax1.axvline(x=i, color='r', linestyle='--')
plt.show()
a,b,c = extract_gait_cycles(group1)
c.describe()

In [ ]:
group1 = accel_data.groupby(by="JointID", as_index=False).get_group(53)
data = extract_gait_cycles(group1, raw=True)
new_dat = data.loc[(data['cycle_label']>212)&(data['cycle_label']<217),["A_tot","cycle_label","salience_points","RollSmooth_A_tot"]]
start_indices = new_dat[new_dat['salience_points']==1].index.tolist()
ax1 = new_dat[["A_tot", "RollSmooth_A_tot"]].plot()
for i in start_indices:
    ax1.axvline(x=i, color='r', linestyle='--')
plt.show()
a,b,c = extract_gait_cycles(group1)
print a.shape
print b.shape
print c.shape
c.describe()

In [ ]:
gait_cycles = new_dat.groupby("cycle_label")
cycles = [gait_cycles.get_group(grp)["RollSmooth_A_tot"].as_matrix().flatten() for grp in gait_cycles.groups]
cycles = [(cycle-cycle.min())/(cycle.max()-cycle.min()) for cycle in cycles]
polynomials = [interp1d(np.linspace(0,1,len(cycle)),cycle) for cycle in cycles]
# for cycle in cycles[5:10]:
#     plt.plot(np.linspace(0,1,len(cycle)),cycle)
for poly in polynomials:
    plt.plot(np.linspace(0,1,51), poly(np.linspace(0,1,51)), 'b-', alpha=.05)

In [ ]:
roll_group = group1.rolling(center=True, window=int(.2*group1.Samplerate[1]), min_periods=10, win_type="gaussian").mean(std=2)
group1.loc[start_plot:start_plot+diff,"A_tot"].plot()
roll_group.loc[start_plot:start_plot+diff,"A_tot"].plot(color='red')

def rolling_smooth(data, cols, window_proportion=.1, min_samples=4):
    stdev = data.Samplerate/100.
    for col in cols:
        data["RollSmooth_"+col] = data[col].rolling(center=True, window=window_proportion*data.loc[0,'Samplerate'], 
                                                    min_periods=min_samples, win_type='gaussian').mean(std=stdev)
    return data

### Figure out how to trim off data that doesn't actually contain walking data

In [ ]:

        


dft = fft(group1.A_tot)
# x_vals = np.arange(1,len(dft)+1).astype(float)
# x_vals = x_vals*group1.Samplerate[1]/len(dft)
num_coefs = group1.shape[0]//20
start_plot = 10000
diff = 2000
start = dft[:num_coefs]
end = dft[-num_coefs:]
dft = np.zeros_like(dft)
dft[:num_coefs] = start
dft[-num_coefs:] = end
# plt.plot(dft)
# plt.show()
smoothed_signal = ifft(dft)
plt.plot(group1.loc[start_plot:start_plot+diff,"A_tot"].as_matrix())
# plt.show()
plt.plot(smoothed_signal[start_plot:start_plot+diff])

## Find Min Salience Vector as described in literature

In [ ]:
roll_group = group1.rolling(center=True, window=int(.5*group1.Samplerate[1]), min_periods=10).min()
roll_group.loc[200000:202000,"A_tot"].plot()
group1.loc[200000:202000,"A_tot"].plot()

In [ ]:


    

#     std_dev = data[col].std()
#     new_col = col+"_rolling_std"
#     data[new_col] = data[col].rolling(center=True, window=int(data.Samplerate[1]), min_periods=data.Samplerate[0]//2).std()
#     data.loc[data[new_col]<std_dev/20,new_col] = -1
#     data.loc[data[new_col]<0, col] = -10000

#     data = data.copy()
#     data["smoothed"] = wavelet_smooth(data, [col])
#     std_dev = data["smoothed"].std()
#     new_col = col+"_rolling_std"
#     data[new_col] = data['smoothed'].rolling(center=True, window=int(data.Samplerate[1]), min_periods=data.Samplerate[0]//2).std()
#     data.loc[data[new_col]<std_dev/15,"smoothed"] = -1
#     data[new_col] = data['smoothed'].rolling(center=True, window=int(data.Samplerate[1]), min_periods=data.Samplerate[0]//2).min()
#     data.loc[data[new_col]<0, col] = -10000

In [ ]:
function_test = get_rid_of_useless_data(group1)

In [ ]:
function_test[-1000:]

In [ ]:
function_test.A_tot.plot()
plt.show()
group1.A_tot.plot()

In [ ]:
def segment_gait_cycles(data):
    pass